In [1]:
import os 
import sys 
from SourceFiles.teproteus import TEProteusAdmin as TepAdmin
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# connect and initialize
admin = TepAdmin() #required to control PXI module
sid = 8 #PXI slot WDS found
inst = admin.open_instrument(slot_id=sid)
resp = inst.send_scpi_query("*IDN?")
print('connected to: ' + resp) # Print *IDN
inst.send_scpi_cmd('*CLS; *RST')

Load TEProteus.dll from : C:\Windows\system32\TEProteus.dll
connected to: Tabor Electronics,P9484M,000002243278,1.237.0 --slot#: 8,


0

In [3]:
# initialize DAC
sampleRateDAC = 1E9
inst.send_scpi_cmd(':FREQ:RAST {0}'.format(sampleRateDAC))

0

In [4]:
# initialize EXT trigger (trigger for every channel need to be set independantly!)
# TRIG:LEV property of a trigger
# TRIG:STAT:SOUR:ENAB property of chan
# TRIG:STAT property of a selected trigger to an selected channel

inst.send_scpi_cmd(':INST:CHAN 1') 
inst.send_scpi_cmd(':TRIG:SOUR:ENAB TRG1')
inst.send_scpi_cmd(':TRIG:SEL TRG1')
inst.send_scpi_cmd(':TRIG:LEV 0.1')
inst.send_scpi_cmd(':TRIG:STAT ON')

inst.send_scpi_cmd(':INST:CHAN 2') 
inst.send_scpi_cmd(':TRIG:SOUR:ENAB TRG1')
inst.send_scpi_cmd(':TRIG:STAT ON')



0

In [5]:
# create waveform
amp = 1  
cycles = 10
segLen = 1024 # must be a multiple of 64
time = np.linspace(0, segLen-1, segLen)
w = 2 * np.pi 
dacWave = amp * np.sin(w*time*cycles/segLen) 
max_dac=65535 # Max Dac
half_dac=max_dac/2 # DC Level
data_type = np.uint16 # DAC data type 
dacWave = ((dacWave) + 1.0) * half_dac  
dacWave = dacWave.astype(data_type) 

In [6]:
# select CH1
ch = 1 # everything after relates to CH 1 
segnum = 1
inst.send_scpi_cmd(':INST:CHAN {0}'.format(ch)) 
inst.send_scpi_cmd(':TRAC:DEL:ALL')
inst.send_scpi_cmd(':TRAC:DEF {0}, {1}'.format(segnum, len(dacWave)))
inst.send_scpi_cmd(':TRAC:SEL {0}'.format(segnum))
inst.write_binary_data(':TRAC:DATA', dacWave) # write, and wait while *OPC completes

inst.send_scpi_cmd(':TASK:COMP:LENG 1')
inst.send_scpi_cmd(':TASK:COMP:SEL 1')
inst.send_scpi_cmd(':TASK:COMP:SEGM 1')

inst.send_scpi_cmd(':TASK:COMP:ENAB TRG1')
inst.send_scpi_cmd(':TASK:COMP:NEXT1 1')
inst.send_scpi_cmd(':TASK:COMP:WRITE')
inst.send_scpi_cmd(':SOUR:FUNC:MODE TASK')
inst.send_scpi_cmd(':OUTP ON')

0

In [7]:
ch = 2 # everything after relates to CH 2
inst.send_scpi_cmd(':INST:CHAN {0}'.format(ch)) 
inst.send_scpi_cmd(':TASK:COMP:LENG 1')
inst.send_scpi_cmd(':TASK:COMP:SEL 1')
inst.send_scpi_cmd(':TASK:COMP:SEGM 1')
inst.send_scpi_cmd(':TASK:COMP:ENAB TRG1')
inst.send_scpi_cmd(':TASK:COMP:NEXT1 1')
inst.send_scpi_cmd(':TASK:COMP:WRITE')
inst.send_scpi_cmd(':SOUR:FUNC:MODE TASK')
inst.send_scpi_cmd(':OUTP ON')

0

0